In [1]:
import numpy as np
import pandas as pd
import pandas as pd
import re
from konlpy.tag import Okt
import pandas as pd
import re

market_df = pd.read_csv('Maket_kurly.csv', encoding = 'utf-8')
market_df.head()

,title,nickname,datetime,content
0,마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴하게!,고미리레,2023. 5. 22. 12:24,리뷰\n마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴...
1,마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔 보세요.,코나맘,2023. 5. 24. 23:38,식품 리뷰\n마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔...
2,자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추천,영앤지,2023. 8. 29. 23:02,[블챌] 체크인 챌린지\n자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추...
3,낭만어시장 땡초넣은 모듬 조개탕 마켓컬리,단감,2023. 8. 29. 13:29,"*내돈내산,직사용후기*\n낭만어시장 땡초넣은 모듬 조개탕 마켓컬리\n단감 ・ 202..."
4,마켓컬리 첫구매 신규가입 쿠폰 추천상품,행복바이러스,2023. 2. 24. 9:20,엄마의 장바구니\n마켓컬리 첫구매 신규가입 쿠폰 추천상품\n행복바이러스 ・ 2023...


In [2]:
def preprocess_text(text):
    # 1. 불필요한 메타데이터 제거
    text = text.split('인쇄')[0]

    # 2. 줄바꿈 및 들여쓰기 제거
    text = text.replace('\n', ' ').strip()

    # 3. 텍스트 정규화
    text = text.lower() # 모두 소문자로 변환
    text = re.sub(r'[^\w\s]', ' ', text) # 문장 부호 및 특수 문자 제거

    # 4. Tokenization and stemming
    okt = Okt()
    tokens = okt.morphs(text)

    # 5. 불용어 제거
    stopwords = ['이', '그', '것', '의', '을', '를', '와', '과']
    filtered_tokens = [word for word in tokens if word not in stopwords]

    return ' '.join(filtered_tokens)

# 해시태그 추출 함수
def extract_hashtags(text):
    hashtags = re.findall(r'\n#\w+', text)
    return hashtags

# 공감 수 추출 함수
def extract_likes(text):
    match = re.search(r'\n공감 (\d+)', text)
    if match:
        return int(match.group(1))
    return None

# 댓글 수 추출 함수
def extract_comments(text):
    match = re.search(r'\n댓글 (\d+)', text)
    if match:
        return int(match.group(1))
    return None

# 해시태그 칼럼 전처리
def hashtag_split(text):
    text = text.replace('\n#','')
    return text

def del_oddsments(list_):
    new_list = list(map(hashtag_split, list_))
    return new_list

# 새로운 칼럼 추가
market_df['hashtags'] = market_df['content'].apply(extract_hashtags)
market_df['likes'] = market_df['content'].apply(extract_likes)
market_df['comments'] = market_df['content'].apply(extract_comments)
# 결측치 처리 및 해시태그 '\n#' 삭제
market_df['hashtags'] = market_df['hashtags'].apply(del_oddsments)
market_df['likes'] = market_df['likes'].fillna(0)
market_df['comments'] = market_df['comments'].fillna(0)

market_df['preprocessed_content'] = market_df['content'].apply(preprocess_text)

In [3]:
market_df.head()

,title,nickname,datetime,content,hashtags,likes,comments,preprocessed_content
0,마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴하게!,고미리레,2023. 5. 22. 12:24,리뷰\n마켓컬리 8주년 위크 역대급 특가 추천 상품 포도 계란 소고기 마트보다 저렴...,"[마켓컬리8주년위크, 마켓컬리, 컬리8주년, 마켓컬리할인, 마켓컬리추천템, 컬리추천...",47.0,9.0,리뷰 마켓 컬리 8 주년 위 크 역대 급 특 가 추천 상품 포도 계란 소고기 마트 ...
1,마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔 보세요.,코나맘,2023. 5. 24. 23:38,식품 리뷰\n마켓컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억의 국민학교 떡볶이 드셔...,"[마켓컬리, 마켓컬리, 오늘도컬리, ID_tkdlqlek]",4.0,2.0,식품 리뷰 마켓 컬리 떡볶이 분식집 떡볶이 좋아하신다면 추억 국민학교 떡볶이 드셔 ...
2,자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추천,영앤지,2023. 8. 29. 23:02,[블챌] 체크인 챌린지\n자취생 재주문 마켓컬리 헤비유저의 실패하지 않는 밀키트 추...,"[마켓컬리추천템, 윅셔너리뱡뱡면, 뱡뱡면, 금미옥, 금미옥닭볶이, 마켓컬리, 자취생...",44.0,39.0,블챌 체크 인 챌 린지 자취 생 재 주문 마켓 컬리 헤비 유저 실패하지 않는 밀 키...
3,낭만어시장 땡초넣은 모듬 조개탕 마켓컬리,단감,2023. 8. 29. 13:29,"*내돈내산,직사용후기*\n낭만어시장 땡초넣은 모듬 조개탕 마켓컬리\n단감 ・ 202...","[모듬조개탕, 조개탕밀키트, 마켓컬리조개탕, 마켓컬리추천, 땡초넣은모듬조개탕, 땡초...",4.0,2.0,내 돈 내 산 직 사용 후기 낭만 어시장 땡초 넣은 모듬 조개 탕 마켓 컬리 단감 ...
4,마켓컬리 첫구매 신규가입 쿠폰 추천상품,행복바이러스,2023. 2. 24. 9:20,엄마의 장바구니\n마켓컬리 첫구매 신규가입 쿠폰 추천상품\n행복바이러스 ・ 2023...,"[마켓컬리, 홍루이젠, 도산분식, 포켓몬빵, 대구반할만떡, 와코사각유부, 홍루이젠,...",8.0,2.0,엄마 장바구니 마켓 컬리 첫 구매 신규가입 쿠폰 추천 상품 행복 바이러스 2023 ...


In [4]:
market_df.to_csv('preprocessed_content_market.csv', encoding='utf-8', index=False)